In [ ]:
import pandas as pd
import numpy as np
import gc
import time

df=pd.read_csv("df.csv")

In [ ]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
del df
gc.collect()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

X = train_df[feats]
y = train_df['TARGET']

total = X.isnull().sum().sort_values(ascending = False)
percent = (X.isnull().sum() / X.isnull().count()*100).sort_values(ascending = False)
missing_X = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

L=missing_X[missing_X['Total']==0]
L1=L.reset_index().values
nomiss_feats=[]
for e in range(0, L.shape[0]):
    nomiss_feats.append(L1[e][0])

del missing_X
del L
del L1
gc.collect()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

for f in X.columns:
    if f in nomiss_feats:
        X[f] = scaler.fit_transform(X[f].values.reshape(-1,1))
        
y = scaler.fit_transform(y.values.reshape(-1,1))

del nomiss_feats
gc.collect()

X = X.fillna(value = 0)

# split data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.2)#, random_state=42)

print (X_train.shape)
print (X_valid.shape)
print (y_train.shape)
print (y_valid.shape)

In [ ]:
# Build and run model
import tensorflow as tf
ITERATIONS = 40000
LEARNING_RATE = 1e-4

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
 
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Let's train the model
feature_count = X_train.shape[1]
x = tf.placeholder('float', shape=[None, feature_count], name='x')
y_ = tf.placeholder('float', shape=[None, 1], name='y_')

print(x.get_shape())

nodes = 20

w1 = weight_variable([feature_count, nodes])
b1 = bias_variable([nodes])
l1 = tf.nn.relu(tf.matmul(x, w1) + b1)

w2 = weight_variable([nodes, 1])
b2 = bias_variable([1])
y = tf.nn.sigmoid(tf.matmul(l1, w2) + b2)

cross_entropy = -tf.reduce_mean(y_*tf.log(tf.maximum(0.00001, y)) + (1.0 - y_)*tf.log(tf.maximum(0.00001, 1.0-y)))
reg = 0.01 * (tf.reduce_mean(tf.square(w1)) + tf.reduce_mean(tf.square(w2)))

predict = (y > 0.5)

correct_prediction = tf.equal(predict, (y_ > 0.5))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
                              
                              

train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy + reg)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for i in range(ITERATIONS):
    feed={x:X_train, y_:y_train}
    sess.run(train_step, feed_dict=feed)
    if i % 1000 == 0 or i == ITERATIONS-1:
        print('{} {} {:.2f}%'.format(i, sess.run(cross_entropy, feed_dict=feed), sess.run(accuracy, feed_dict=feed)*100.0))


In [ ]:
T = test_df[feats]

total = T.isnull().sum().sort_values(ascending = False)
percent = (T.isnull().sum() / T.isnull().count()*100).sort_values(ascending = False)
missing_T = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

L2=missing_T[missing_T['Total']==0]
L3=L2.reset_index().values
nomiss_feats=[]
for e in range(0, L2.shape[0]):
    nomiss_feats.append(L3[e][0])
    
del missing_T
del L2
del L3
gc.collect()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

for f in T.columns:
    if f in nomiss_feats:
        T[f] = scaler.fit_transform(T[f].values.reshape(-1,1))
        
del nomiss_feats
gc.collect()

T